## Notebook for making predictions with deleted constant y samples


In [21]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

### Reading dataset stored in other notebooks

In [22]:
sys.path.append('../')

from data_prep import dm_saver, data_manager

dms = dm_saver.DM_Saver.load('../dms')

dml = data_manager.Data_Manager()

dml.dms2dm(dms)
%store dml

Stored 'dml' (Data_Manager)


In [23]:


%store -r dml

dm = dml

## Getting data

In [24]:
print(dm.data_A.columns)

X_A_train = dm.data_A.iloc[:, 2:]
Y_A_train = dm.data_A.iloc[:, 1]

X_B_train = dm.data_B.iloc[:, 2:]
Y_B_train = dm.data_B.iloc[:, 1]

X_C_train = dm.data_C.iloc[:, 2:]
Y_C_train = dm.data_C.iloc[:, 1]

X_A_test = dm.X_test_estimated_a[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_B_test = dm.X_test_estimated_b[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_C_test = dm.X_test_estimated_c[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]

#X_A_train['direct_rad:W_lag_12']

Index(['date_forecast', 'pv_measurement', 'absolute_humidity_2m:gm3',
       'air_density_2m:kgm3', 'clear_sky_energy_1h:J', 'clear_sky_rad:W',
       'dew_or_rime:idx', 'dew_point_2m:K', 'diffuse_rad:W',
       'diffuse_rad_1h:J', 'direct_rad:W', 'direct_rad_1h:J',
       'effective_cloud_cover:p', 'elevation:m', 'fresh_snow_12h:cm',
       'fresh_snow_1h:cm', 'fresh_snow_24h:cm', 'fresh_snow_3h:cm',
       'fresh_snow_6h:cm', 'is_day:idx', 'is_in_shadow:idx',
       'msl_pressure:hPa', 'precip_5min:mm', 'precip_type_5min:idx',
       'pressure_100m:hPa', 'pressure_50m:hPa', 'prob_rime:p',
       'rain_water:kgm2', 'relative_humidity_1000hPa:p', 'sfc_pressure:hPa',
       'snow_depth:cm', 'snow_drift:idx', 'snow_melt_10min:mm',
       'snow_water:kgm2', 'sun_azimuth:d', 'sun_elevation:d',
       'super_cooled_liquid_water:kgm2', 't_1000hPa:K', 'total_cloud_cover:p',
       'visibility:m', 'wind_speed_10m:ms', 'wind_speed_u_10m:ms',
       'wind_speed_v_10m:ms', 'wind_speed_w_1000hPa:m

## Catboost models
will try with two models:
-   One for dataset A
-   One for B and C

In [25]:
catModel_a = CatBoostRegressor()
catModel_b = CatBoostRegressor()
catModel_c = CatBoostRegressor()

catModel_bc = CatBoostRegressor()

#catModel_abc = CatBoostRegressor()

In [26]:
X_BC_train = pd.concat([X_B_train, X_C_train])
Y_BC_train = pd.concat([Y_B_train, Y_C_train])

X_ABC_train = pd.concat([X_A_train, X_BC_train])
Y_ABC_train = pd.concat([Y_A_train, Y_BC_train])

In [27]:
catModel_a.fit(X_A_train,Y_A_train)
catModel_b.fit(X_B_train,Y_B_train)
catModel_c.fit(X_C_train,Y_C_train)

#catModel_bc.fit(X_BC_train,Y_BC_train)

#catModel_abc.fit(X_ABC_train,Y_ABC_train)

Learning rate set to 0.007444
0:	learn: 1305.0865181	total: 15.1ms	remaining: 3m 46s
1:	learn: 1297.7801317	total: 21.9ms	remaining: 2m 44s
2:	learn: 1290.5776300	total: 29.3ms	remaining: 2m 26s
3:	learn: 1283.2575400	total: 44.8ms	remaining: 2m 47s
4:	learn: 1276.1949154	total: 59.2ms	remaining: 2m 57s
5:	learn: 1269.0307310	total: 68ms	remaining: 2m 49s
6:	learn: 1261.8874941	total: 75.3ms	remaining: 2m 41s
7:	learn: 1254.9937362	total: 82.5ms	remaining: 2m 34s


8:	learn: 1248.0138671	total: 90.5ms	remaining: 2m 30s
9:	learn: 1241.1564153	total: 97.6ms	remaining: 2m 26s
10:	learn: 1234.2579788	total: 105ms	remaining: 2m 22s
11:	learn: 1227.4957939	total: 112ms	remaining: 2m 20s
12:	learn: 1220.8235376	total: 119ms	remaining: 2m 17s
13:	learn: 1214.1031350	total: 128ms	remaining: 2m 17s
14:	learn: 1207.4376282	total: 138ms	remaining: 2m 18s
15:	learn: 1201.1089922	total: 146ms	remaining: 2m 16s
16:	learn: 1194.6371799	total: 153ms	remaining: 2m 14s
17:	learn: 1188.3289527	total: 160ms	remaining: 2m 13s
18:	learn: 1181.8965501	total: 167ms	remaining: 2m 11s
19:	learn: 1175.6833105	total: 175ms	remaining: 2m 10s
20:	learn: 1169.4786225	total: 182ms	remaining: 2m 9s
21:	learn: 1163.2690286	total: 188ms	remaining: 2m 8s
22:	learn: 1157.1237547	total: 195ms	remaining: 2m 7s
23:	learn: 1150.9272966	total: 202ms	remaining: 2m 5s
24:	learn: 1145.0228796	total: 208ms	remaining: 2m 4s
25:	learn: 1139.1158791	total: 216ms	remaining: 2m 4s
26:	learn: 1133.

In [28]:
# Feature importance
catModel_a.get_feature_importance(prettified=True)

,Feature Id,Importances
0,direct_rad:W,24.618077
1,diffuse_rad:W,5.994591
2,sun_elevation:d,5.776023
3,direct_rad_1h:J,5.721174
4,clear_sky_rad:W,5.065020
5,sun_azimuth:d,4.990570
6,day_year,4.457568
7,wind_speed_u_10m:ms,3.716617
8,direct_rad:W_lag_-23,3.557475
9,diffuse_rad_1h:J,2.927861


## Preds

In [29]:
pred_A = catModel_a.predict(X_A_test)
pred_B = catModel_b.predict(X_B_test)
pred_C = catModel_c.predict(X_C_test)

# pred_B = catModel_bc.predict(X_B_test)
# pred_C = catModel_bc.predict(X_C_test)

# pred_A = catModel_abc.predict(X_A_test)
# pred_B = catModel_abc.predict(X_B_test)
# pred_C = catModel_abc.predict(X_C_test)

In [30]:
df_A = pd.DataFrame()

df_A["predict"] = pred_A
df_A["location"] = "A"

df_B = pd.DataFrame()

df_B["predict"] = pred_B
df_B["location"] = "B"

df_C = pd.DataFrame()

df_C["predict"] = pred_C
df_C["location"] = "C"

df_mid = pd.concat([df_A, df_B], ignore_index=True)

df = pd.concat([df_mid, df_C], join="inner", ignore_index=True)

df = df.drop("location", axis=1)

df[df<0] = 0

# NAME THE FILE 
df.to_csv("./ext_preds/bestpred_15k.csv")

df.shape

(2160, 1)